In [214]:
import pandas as pd
import numpy as np
import string
import json
table = str.maketrans('', '', string.punctuation)
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from gensim.parsing.preprocessing import remove_stopwords
import re
import pickle
DocumentFreq = pickle.load( open( "DocumentFreq.pickle", "rb" ) )

In [215]:
def word_count_pers(str,wordy):
    count=0
    str=remove_stopwords(str)
    wordy = wordnet_lemmatizer.lemmatize(wordy)
    words = word_tokenize(str)
    for word in words:
        word = wordnet_lemmatizer.lemmatize(word)
        if word==wordy:
            count+=1
    return count 

def pl(passage):
    passage=re.sub(r'-', ' ', passage)
    passage=passage.translate(table).lower()
    passage=re.sub(r'\n', ' ', passage)
    passage=remove_stopwords(passage)
    words = word_tokenize(passage)
    return (len(words))

In [216]:
with open('D:/Learning Material/IR/WikiPassageQA/data/raw/WikiPassageQA/document_passages.json') as f:
    data = json.load(f)

In [217]:
df = pd.read_csv("D:/Learning Material/IR/WikiPassageQA/data/raw/extracted_query_data/train_exp.csv")

In [218]:
df

,DocumentID,DocumentName,QID,Question,RelevantPassages
0,672,Evangelicalism.html,3086,What is the role of conversionism in Evangelic...,4
1,359,French_Revolution.html,195,How did the assault on the Bastille the first ...,1
2,359,French_Revolution.html,195,How did the assault on the Bastille the first ...,2
3,285,Albania.html,557,What is the prehistory of Albania?,4
4,579,Central_Powers.html,1508,What significance did Bulgaria have in the end...,14
...,...,...,...,...,...
5555,641,Ancient_Egypt.html,1971,How did conquering of Alexander the great effe...,29
5556,641,Ancient_Egypt.html,1971,How did conquering of Alexander the great effe...,30
5557,740,Amnesty_International.html,1153,What is Amnesty International?,0
5558,740,Amnesty_International.html,1153,What is Amnesty International?,9


In [219]:
allpassages=[]
for i in data.keys():
    for j in data[i].keys():
        allpassages.append(data[i][j])
N=len(allpassages)
print(N)

passage_lengths=[]
for i in allpassages:
    passage=re.sub(r'-', ' ', passage)
    passage=i.translate(table).lower()
    passage=re.sub(r'\n', ' ', passage)
    passage=remove_stopwords(passage)
    words = word_tokenize(passage)
    passage_lengths.append(len(words))
avdl = np.array(passage_lengths).mean()
print(avdl)

50612
74.06235675333913


In [220]:
uniqueQids= df['QID'].unique().tolist()
len(uniqueQids)

3332

In [221]:
relpassages=dict()
for i in uniqueQids:
    relpassages[i]=[]
    
for i in uniqueQids:
    for j in df.loc[df["QID"]==i].values:
        temp=[j[0],j[4]]
        relpassages[i].append(temp)
    

In [222]:
(relpassages)

{3086: [[672, 4]],
 195: [[359, 1], [359, 2]],
 557: [[285, 4]],
 1508: [[579, 14]],
 956: [[204, 9]],
 1993: [[2, 16]],
 2260: [[430, 10], [430, 11]],
 2678: [[341, 7], [341, 8]],
 2165: [[420, 22], [420, 23]],
 2079: [[561, 15]],
 1287: [[765, 0]],
 2958: [[189, 20], [189, 21]],
 4002: [[817, 4], [817, 5]],
 1634: [[328, 6]],
 2610: [[97, 6], [97, 7]],
 3381: [[742, 26], [742, 27]],
 3534: [[603, 16], [603, 17], [603, 18], [603, 19]],
 511: [[788, 0]],
 1824: [[6, 4]],
 25: [[135, 16], [135, 17], [135, 18]],
 75: [[512, 54], [512, 55], [512, 56]],
 1435: [[263, 4], [263, 5]],
 1545: [[462, 29]],
 1130: [[598, 8], [598, 9]],
 1372: [[348, 12]],
 458: [[478, 0]],
 2450: [[547, 29]],
 3203: [[707, 7]],
 1831: [[59, 2]],
 4033: [[353, 1]],
 3565: [[600, 2], [600, 3], [600, 4]],
 2938: [[757, 3], [757, 4]],
 2822: [[480, 0]],
 3295: [[700, 26], [700, 27], [700, 28]],
 1205: [[206, 3]],
 1705: [[807, 14], [807, 15]],
 3318: [[852, 24], [852, 25], [852, 26]],
 2497: [[23, 7], [23, 8], [23, 

In [223]:
qID= 3047
question=df.loc[df["QID"]==qID]["Question"].values[0]
question

'What is the etymology of Suriname?'

In [224]:
question=re.sub(r'-', ' ', question)
question=question.translate(table).lower()
question=remove_stopwords(question)
question_words= word_tokenize(question)
question_words

['etymology', 'suriname']

In [225]:
ri=[]
for i in question_words:
    r=0
    for k in relpassages[qID]:
        tdocid=k[0]
        trelp =k[1]
        tpass = re.sub(r'-', ' ', data[str(tdocid)][str(trelp)])
        tpass = re.sub(r'\n', ' ',tpass.translate(table).lower())
        tpass = re.sub(r'\n', ' ', data[str(tdocid)][str(trelp)].translate(table).lower())
        if i in tpass:
            r+=1
    ri.append(r)  
ri

[0, 3]

In [213]:

allpassages=[]
for i in data.keys():
    for j in data[i].keys():
        allpassages.append(data[i][j])
        
refinedpassages=[]
for i in allpassages:
    passage=re.sub(r'-', ' ',i)
    passage=passage.translate(table).lower()
    passage=re.sub(r'\n', ' ', passage)
    refinedpassages.append(passage)
    
ni=[]   
for i in question_words:
    count=0
    for passages in refinedpassages:
        if i in passages:
            count+=1
            continue
    ni.append(count)
ni    

[66, 86]

In [175]:
R = len(relpassages[qID])
R

3

In [226]:
qi=[]
for i in question_words:
    qi.append(word_count_pers(question,i))
    
print(qi)
"""
di=[]
for i in question_words:
    di.append(word_count_pers(passage,i))
    
print(di)
"""

[1, 1]


'\ndi=[]\nfor i in question_words:\n    di.append(word_count_pers(passage,i))\n    \nprint(di)\n'

In [231]:
dl=pl(passage)
dl

134

In [232]:
k1=1.2
k2=500
b=0.75
K = k1*((1-b) + b*(dl/avdl))
K

1.9283575798384607

In [233]:
BMscore=0
for i,j,k,l in zip(ri,ni,qi,di):
    a=(i+0.5)/(R-i+0.5)
    #print(a)
    b=(j-i+0.5)/(N-j-R+i+0.5)
    #print(b)
    c=(k1+1)*l/K+l
    #print(c)
    d=(k2+1)*k/k2+k
    #print(d)
    e=a/b
    #print(e)
    score= np.log(e)*c*d
    #print(score)
    BMscore+=score

In [234]:
BMscore

35.79387907925841

In [ ]:
BMsp=[]
for i in allpassages:
    dl=pl(i)
    di=[]
    for w in question_words:
        di.append(word_count_pers(i,w))
    BMscore=0
    for i,j,k,l in zip(ri,ni,qi,di):
        a=(i+0.5)/(R-i+0.5)
        #print(a)
        b=(j-i+0.5)/(N-j-R+i+0.5)
        #print(b)
        c=(k1+1)*l/K+l
        #print(c)
        d=(k2+1)*k/k2+k
        #print(d)
        e=a/b
        #print(e)
        score= np.log(e)*c*d
        #print(score)
        BMscore+=score    
    BMsp.append(BMscore)    

In [118]:
len(BMsp)

50612

In [119]:
BMsp=(np.array(BMsp))

sortedindices=BMsp.argsort()[-20:][::-1]

sortedindices

array([13220, 46193, 40698,  4761, 39194, 40196, 19874, 16404,  8247,
        5882,  5880, 20524, 13699, 21938,  3171, 39184, 42349, 34590,
       22379, 32240], dtype=int64)

In [120]:
for i in sortedindices:
    print(BMsp[i])

120.71737321406938
120.71737321406938
78.72130614140913
72.62250092320771
61.31907158086506
61.31907158086506
60.35868660703469
60.35868660703469
60.35868660703469
60.35868660703469
60.35868660703469
49.05525726469205
49.05525726469205
36.791442948519034
36.791442948519034
36.791442948519034
36.791442948519034
36.791442948519034
36.791442948519034
36.791442948519034


In [93]:
passage=allpassages[32240]

In [95]:
passage=re.sub(r'-', ' ', passage)
passage=passage.translate(table).lower()
passage=re.sub(r'\n', ' ', passage)
passage

'after the dissolution of the tokugawa shogunate the role of the commander in chief resided with the emperor of japan the present day constitutional role of the emperor is that of a ceremonial figurehead without any military role after japans move towards democracy the position of commander in chief of the japanese self defence forces is held by the prime minister of japan military authority runs from the prime minister to the cabinet level minister of defense of the japanese ministry of defense the constitution of the netherlands states in article 97 that the government shall have supreme authority over the armed forces article 42 defines the government as the monarch and the ministers and that only ministers are responsible for acts of government'

In [121]:
allpassages[0]

'Townspeople who lived in chartered towns were burghers, as opposed to serfs who lived in villages. Towns were often "free", in the sense that they were directly protected by the king or emperor, and were not part of a feudal fief. [citation needed]\n\nToday the process for granting charters is determined by the type of government of the state in question. In monarchies, charters are still often a royal charter given by the Crown or the state authorities acting on behalf of the Crown. In federations, the granting of charters may be within the jurisdiction of the lower level of government such as a state or province. [citation needed]\n\nIn Brazil, municipal corporations are called municipios and are created by means of local legislation at the state level, or after passing a referendum vote of the affected population.'